In [1]:
import torch
import numpy as np
import pandas as pd
import matchzoo as mz
print('matchzoo version', mz.__version__)

matchzoo version 1.2


In [2]:
ranking_task = mz.tasks.Ranking(losses=mz.losses.RankCrossEntropyLoss(num_neg=1))
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]
print("`ranking_task` initialized with metrics", ranking_task.metrics)

`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]


In [3]:
print('data loading ...')
train_pack_raw = mz.datasets.wiki_qa.load_data('train', task=ranking_task)
dev_pack_raw = mz.datasets.wiki_qa.load_data('dev', task=ranking_task, filtered=True)
test_pack_raw = mz.datasets.wiki_qa.load_data('test', task=ranking_task, filtered=True)
print('data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`')

data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`


In [4]:
preprocessor = mz.models.aNMM.get_default_preprocessor()

In [5]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 3804.20it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:08<00:00, 2240.37it/s]
Building Vocabulary from a datapack.: 100%|██████████| 418401/418401 [00:00<00:00, 768049.52it/s] 
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 7258.07it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:08<00:00, 2280.20it/s]
Processing length_right with len: 100%|██████████| 18841/18841 [00:00<00:00, 778209.91it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 4621.41it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00<00

In [6]:
preprocessor.context

{'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x7f1e30d91978>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7f1e30d91ef0>,
 'vocab_size': 30058,
 'embedding_input_dim': 30058}

In [7]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=300)
term_index = preprocessor.context['vocab_unit'].state['term_index']
embedding_matrix = glove_embedding.build_matrix(term_index)
l2_norm = np.sqrt((embedding_matrix * embedding_matrix).sum(axis=1))
embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [8]:
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=2,
    num_neg=1,
    batch_size=20,
    resample=True,
    sort=False,
    shuffle=True
)
testset = mz.dataloader.Dataset(
    data_pack=test_pack_processed,
    batch_size=20,
    sort=False,
    shuffle=False
)

In [9]:
padding_callback = mz.models.aNMM.get_default_padding_callback()

trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    stage='train',
    callback=padding_callback,
)
testloader = mz.dataloader.DataLoader(
    dataset=testset,
    stage='dev',
    callback=padding_callback,
)

In [10]:
model = mz.models.aNMM()

model.params['task'] = ranking_task
model.params['embedding'] = embedding_matrix

model.params['dropout_rate'] = 0.1

model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

aNMM(
  (embedding): Embedding(30058, 300, padding_idx=0)
  (matching): Matching()
  (hidden_layers): Sequential(
    (0): Sequential(
      (0): Linear(in_features=200, out_features=100, bias=True)
      (1): ReLU()
    )
    (1): Sequential(
      (0): Linear(in_features=100, out_features=1, bias=True)
      (1): ReLU()
    )
  )
  (q_attention): Attention(
    (linear): Linear(in_features=300, out_features=1, bias=False)
  )
  (dropout): Dropout(p=0.1, inplace=False)
  (out): Linear(in_features=1, out_features=1, bias=True)
)
Trainable params:  9037903


In [11]:
optimizer = torch.optim.Adam(model.parameters(), lr = 3e-4)

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=15
)

trainer.run()

[Iter-102 Loss-0.693]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6327 - normalized_discounted_cumulative_gain@5(0.0): 0.6798 - mean_average_precision(0.0): 0.6359



[Iter-204 Loss-0.693]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6311 - normalized_discounted_cumulative_gain@5(0.0): 0.6782 - mean_average_precision(0.0): 0.6338



[Iter-306 Loss-0.693]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6311 - normalized_discounted_cumulative_gain@5(0.0): 0.6782 - mean_average_precision(0.0): 0.6338



[Iter-408 Loss-0.693]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6311 - normalized_discounted_cumulative_gain@5(0.0): 0.6782 - mean_average_precision(0.0): 0.6338



[Iter-510 Loss-0.693]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6311 - normalized_discounted_cumulative_gain@5(0.0): 0.6782 - mean_average_precision(0.0): 0.6338



...

[Iter-1122 Loss-0.693]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6311 - normalized_discounted_cumulative_gain@5(0.0): 0.6782 - mean_average_precision(0.0): 0.6338



[Iter-1224 Loss-0.693]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6311 - normalized_discounted_cumulative_gain@5(0.0): 0.6782 - mean_average_precision(0.0): 0.6338



[Iter-1326 Loss-0.693]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6311 - normalized_discounted_cumulative_gain@5(0.0): 0.6782 - mean_average_precision(0.0): 0.6338



[Iter-1428 Loss-0.693]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6311 - normalized_discounted_cumulative_gain@5(0.0): 0.6782 - mean_average_precision(0.0): 0.6338



[Iter-1530 Loss-0.693]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6311 - normalized_discounted_cumulative_gain@5(0.0): 0.6782 - mean_average_precision(0.0): 0.6338

Cost time: 400.23424100875854s
